In [1]:
#Hasan Audio classification code
import os
import wave
import numpy as np
from PIL import Image
from pydub import AudioSegment
from scipy.io import wavfile
import random
#import matplotlib as plt
import sys
import io
import glob
import IPython
import wave
import pandas as pd
import pylab as plt
from os import listdir
from os.path import isfile, join

from keras.models import Sequential
from keras.utils import plot_model
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Embedding
from keras.models import Model, load_model
from keras.layers import Dense, Input, Dropout, LSTM, Activation, Input, Bidirectional
from keras.callbacks import ModelCheckpoint
from keras.models import Model, load_model, Sequential
from keras.layers import Dense, Activation, Dropout, Input, Masking, TimeDistributed, LSTM, Conv1D
from keras.layers import GRU, Bidirectional, BatchNormalization, Reshape
from keras.optimizers import Adam

load_feats_again = "y"
max_time = 0

c:\users\hrafiq\appdata\local\programs\python\python35\lib\site-packages\pydub\utils.py:165: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)
c:\users\hrafiq\appdata\local\programs\python\python35\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# def graph_spectrogram(wav_file):
#     sound_info, frame_rate = get_wav_info(wav_file)
#     pylab.figure(num=None, figsize=(19, 12))
#     pylab.subplot(111)
#     pylab.title('spectrogram of %r' % wav_file)
#     pylab.specgram(sound_info, Fs=frame_rate)
#     pylab.savefig('spectrogram.png')
#     pylab.show()
#     return(sound_info, frame_rate)

# Calculate and plot spectrogram for a wav audio file
def graph_spectrogram(wav_file):
    data, rate = get_wav_info(wav_file)
    nfft = 200 # Length of each window segment
    fs = 8000 # Sampling frequencies
    noverlap = 120 # Overlap between windows
    nchannels = data.ndim
    if nchannels == 1:
        pxx, freqs, bins, im = plt.specgram(data, nfft, fs, noverlap = noverlap)
    elif nchannels == 2:
        pxx, freqs, bins, im = plt.specgram(data[:,0], nfft, fs, noverlap = noverlap)
    return pxx

# Load a wav file
# def get_wav_info(wav_file):
#     rate, data = wavfile.read(wav_file)
#     return rate, data

# Used to standardize volume of audio clip
def match_target_amplitude(sound, target_dBFS):
    change_in_dBFS = target_dBFS - sound.dBFS
    return sound.apply_gain(change_in_dBFS)
    
def get_wav_info(wav_file):
    wav = wave.open(wav_file, 'r')
    frames = wav.readframes(-1)
    sound_info = plt.fromstring(frames, 'int16')
    frame_rate = wav.getframerate()
    wav.close()
    return sound_info, frame_rate

def get_batch_spectrograms(batch):
    out = []
    for b in batch:
        out.append(graph_spectrogram(b))
    return out

def pad_with_zeros(x_arr, max_time):
    x_new_arr = []
    for x in x_arr:
        x_dumm = np.zeros((101, max_time))
        x_dumm[:,0:x.shape[1]] = x
        x_new_arr.append(x_dumm)
    return x_new_arr

def find_max_audio_len(x_arr):
    max_time = 0
    for x in x_arr:
        if x.shape[1] > max_time:
            max_time = x.shape[1]
    return max_time

def reshape_x(x_in, max_time):
    #Create full storage
    x_out = np.zeros((len(x_in), max_time, 101))
    
    #Copy each song's feature to matrix
    for i, x in enumerate(x_in):
        x = np.transpose(x)
        x_out[i,:,:] = x
    #x = np.reshape(x, (len(x), max_time, 101))
    return x_out

def process_audio_in_folder(mypath):
    x_array = []
    y_array = []
    print("Running feature extraction ...")
    folders = [f for f in listdir(mypath) if not isfile(join(mypath, f))]
    for fol in folders:
        folderpath = mypath + "\\" + fol
        for fil in listdir(folderpath):
            if isfile(join(folderpath, fil)):
                file_path = join(folderpath, fil)
                print(file_path)
                audio_feat = graph_spectrogram(file_path)
                #print("Label: " + fol + " File: " + fil +" Features size " + str(audio_feat.shape ))
                x_array.append(audio_feat)
                y_array.append(fol)
    
    #Do One hot of Y labels and return numpy
    pd_y = pd.DataFrame(y_array)
    y_array = pd.get_dummies(pd_y, [0]).values
    return(x_array, y_array)

In [3]:
#Dummy -> Get features of files
feats = get_batch_spectrograms([os.getcwd() + "\\anupam\\HipRock\\1.wav", os.getcwd() + "\\anupam\\HipRock\\2.wav"])
for f in feats:
    print(f.shape)

c:\users\hrafiq\appdata\local\programs\python\python35\lib\site-packages\ipykernel_launcher.py:37: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
c:\users\hrafiq\appdata\local\programs\python\python35\lib\site-packages\matplotlib\axes\_axes.py:7221: RuntimeWarning: divide by zero encountered in log10
  Z = 10. * np.log10(spec)


(101, 8818)
(101, 8818)


In [14]:
if load_feats_again == "y":
    #Extract features of all audio files
    #Main folder -> Sub folders 1, 2 ,3 ( class wise ) -> Files per folder
    x_arr, y_arr = process_audio_in_folder(os.getcwd() + "\Sound-Data\\tr")
    print("Number of files:", len(y_arr))
    
    #Loop at all features and find max length
    max_time = find_max_audio_len(x_arr)
    print("Max timing:" , max_time)
    
    #Pad x_array with zeros till max_length
    x_arr = pad_with_zeros(x_arr, max_time)
    
    #Reshape to matrices
    x_arr = reshape_x(x_arr, max_time)
    
    #Data loaded and reshaped
    print("X: ",x_arr.shape)
    print("Y: ",y_arr.shape)
    
    #Save features as npy files
    np.save(os.getcwd() + "\\x_arr", x_arr)
    np.save(os.getcwd() + "\\y_arr", y_arr)
else:
    x_arr = np.load(os.getcwd() + "\\x_arr.npy")
    y_arr = np.load(os.getcwd() + "\\y_arr.npy")
    
    max_time = x_arr.shape[2]
    
    #Data loaded and reshaped
    print("X: ",x_arr.shape)
    print("Y: ",y_arr.shape)
    print("Number of files:", y_arr.shape[0])
    print("Max timing:" , max_time)

Running feature extraction ...
C:\Users\hrafiq\Documents\HRafiq\hrafiq\Machine_Deep_learning\Deep learning\Audio_Classification\AudioSentimentRNN\Sound-Data\tr\Negative\03-01-05-02-01-02-03.wav
C:\Users\hrafiq\Documents\HRafiq\hrafiq\Machine_Deep_learning\Deep learning\Audio_Classification\AudioSentimentRNN\Sound-Data\tr\Negative\03-01-05-02-01-02-10.wav
C:\Users\hrafiq\Documents\HRafiq\hrafiq\Machine_Deep_learning\Deep learning\Audio_Classification\AudioSentimentRNN\Sound-Data\tr\Negative\03-01-05-02-01-02-19.wav
C:\Users\hrafiq\Documents\HRafiq\hrafiq\Machine_Deep_learning\Deep learning\Audio_Classification\AudioSentimentRNN\Sound-Data\tr\Negative\03-01-05-02-01-02-21.wav


c:\users\hrafiq\appdata\local\programs\python\python35\lib\site-packages\ipykernel_launcher.py:37: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
c:\users\hrafiq\appdata\local\programs\python\python35\lib\site-packages\matplotlib\axes\_axes.py:7221: RuntimeWarning: divide by zero encountered in log10
  Z = 10. * np.log10(spec)


C:\Users\hrafiq\Documents\HRafiq\hrafiq\Machine_Deep_learning\Deep learning\Audio_Classification\AudioSentimentRNN\Sound-Data\tr\Negative\03-01-05-02-02-01-06.wav
C:\Users\hrafiq\Documents\HRafiq\hrafiq\Machine_Deep_learning\Deep learning\Audio_Classification\AudioSentimentRNN\Sound-Data\tr\Negative\03-01-05-02-02-01-20.wav
C:\Users\hrafiq\Documents\HRafiq\hrafiq\Machine_Deep_learning\Deep learning\Audio_Classification\AudioSentimentRNN\Sound-Data\tr\Negative\03-01-05-02-02-02-01.wav
C:\Users\hrafiq\Documents\HRafiq\hrafiq\Machine_Deep_learning\Deep learning\Audio_Classification\AudioSentimentRNN\Sound-Data\tr\Negative\03-01-05-02-02-02-03.wav
C:\Users\hrafiq\Documents\HRafiq\hrafiq\Machine_Deep_learning\Deep learning\Audio_Classification\AudioSentimentRNN\Sound-Data\tr\Negative\03-01-05-02-02-02-06.wav
C:\Users\hrafiq\Documents\HRafiq\hrafiq\Machine_Deep_learning\Deep learning\Audio_Classification\AudioSentimentRNN\Sound-Data\tr\Negative\03-01-05-02-02-02-24.wav
C:\Users\hrafiq\Docume

In [15]:
def create_keras_model(input_shape, numb_labels):
    #Start creating Tensorflow based Keras model
    model = Sequential()

    #Add input layer
    #You don't need to include the batch size here
    #input of LSTM has shape as ( Batch size, Time steps, ....... )
    #Input_shape is the dimension of ( Timesteps, ...... )
    X_input = Input(shape = input_shape)

    # Step 1: CONV layer (≈4 lines)
    X = Conv1D(filters=128, kernel_size=15, strides=4)(X_input)# CONV1D
    X = BatchNormalization()(X)               # Batch normalization
    X = Activation('relu')(X)                 # ReLu activation
    #X = Dropout(0.8)(X)                       # dropout (use 0.8)

    #Add first LSTM layer
    ###Very important -> Return_sequences is used to refer that every LSTM time step will return output or only last one
    X = Bidirectional(LSTM(units=128,return_sequences = True))(X)
    
    #Add dropout 0.5
    #X = Dropout(0.5)(X)
    
    #Add second LSTM layer
    ###Very important -> Return_sequences is used to refer that every LSTM time step will return output or only last one
    X = LSTM(units=128,return_sequences = True)(X)
    
    #Add dropout 0.5
    #X = Dropout(0.5)(X)    

    #Add third LSTM layer
    ###Very important -> Return_sequences is used to refer that every LSTM time step will return output or only last one
    X = LSTM(units=128,return_sequences = False)(X)
    
    #Add dropout 0.5
    #X = Dropout(0.5)(X)    

    #Dense layer with 64 neurons
    X = Dense(64, activation='relu')(X)    
    
    #Dense layer with 32 neurons
    X = Dense(32, activation='relu')(X)        
    
    #Add final softmax layer with 5 possible outputs
    X = Dense(numb_labels, activation='softmax')(X)
    
    # Create Model instance which converts sentence_indices into X.
    model = Model(inputs=X_input, outputs=X)
    
    return model

In [18]:
#Initialize model and show summary
k_model = create_keras_model((None,101), y_arr.shape[1]) #Very important -> Create a dynamic length LSTM network so, max_time replaced with None
k_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         (None, None, 101)         0         
_________________________________________________________________
conv1d_5 (Conv1D)            (None, None, 128)         194048    
_________________________________________________________________
batch_normalization_5 (Batch (None, None, 128)         512       
_________________________________________________________________
activation_5 (Activation)    (None, None, 128)         0         
_________________________________________________________________
bidirectional_5 (Bidirection (None, None, 256)         263168    
_________________________________________________________________
lstm_14 (LSTM)               (None, None, 128)         197120    
_________________________________________________________________
lstm_15 (LSTM)               (None, 128)               131584    
__________

In [19]:
opt = Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, decay=0.01)
k_model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['categorical_accuracy'])

In [20]:
#start training process
k_model.fit(x_arr, y_arr, epochs = 40, batch_size = 2, shuffle=True)

Epoch 1/40
20/20 [==============================] - 21s 1s/step - loss: 0.6987 - categorical_accuracy: 0.4000
Epoch 2/40
20/20 [==============================] - 16s 813ms/step - loss: 0.6879 - categorical_accuracy: 0.7000
Epoch 3/40
20/20 [==============================] - 21s 1s/step - loss: 0.6792 - categorical_accuracy: 0.8000
Epoch 4/40
20/20 [==============================] - 17s 858ms/step - loss: 0.6648 - categorical_accuracy: 0.8000
Epoch 5/40
20/20 [==============================] - 17s 853ms/step - loss: 0.6795 - categorical_accuracy: 0.6000
Epoch 6/40
20/20 [==============================] - 22s 1s/step - loss: 0.6616 - categorical_accuracy: 0.7000
Epoch 7/40
20/20 [==============================] - 17s 853ms/step - loss: 0.6172 - categorical_accuracy: 0.8000
Epoch 8/40
20/20 [==============================] - 18s 875ms/step - loss: 0.5825 - categorical_accuracy: 0.8000
Epoch 9/40
20/20 [==============================] - 23s 1s/step - loss: 0.6639 - categorical_accuracy: 0.